### 4. 명목형 변수 인코딩: 온라인 스토어 고객 행동 분석

### #00. 사전준비: 라이브러리 Import

In [ ]:
from hossam import load_data
from pandas import get_dummies, merge, DataFrame
from sklearn.preprocessing import OneHotEncoder
#import numpy as np

### #00. 사전준비: 데이터 load하기

In [ ]:
cstmer = load_data('online_store_customers')
prchse = load_data('online_store_purchases')
cstmer

In [ ]:
prchse

### #01. 데이터 병합하기

In [ ]:
dfC = cstmer.copy()
dfP = prchse.copy()
dfMerged = merge(
    dfC, dfP, left_index = True, right_on = ['user_id'], how = 'outer'
    )
dfMerged

In [ ]:
dfMerged2 = dfMerged.set_index('user_id')
dfMerged2

### #02. 최빈값으로 결측치 정제하기

In [ ]:
counts_ = dfMerged2['size'].value_counts()
counts_

In [ ]:
dfMerged3 = dfMerged2.fillna(value = 'S')
dfMerged3


In [ ]:
#---------------------강사님의 결측치 정제 방법---------------------
#imr = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
#imr_value = imr.fit_transform(merged_df[['size']])
    # Scikit-Learn으로 항상 뭘 할때는 항상 ↑이차원 형태로 전달해야 됨!

#imr_df = DataFrame(imr_value, index=merged_df.index, columns=['size'])

#merged_copy_df = merged_df.copy()
#merged_copy_df['size'] = imr_df['size']
#--------------------------------------------------------------------

### #03. 더미변수 생성해보기(in gender 변수)

In [ ]:
dfMerged4 = get_dummies(
    dfMerged3, columns=['gender'], drop_first=True, dtype='int'
    )
dfMerged4.head()
# 음.. 더미변수 생성은 일반통계모형에서 판다스 이용해서 
# 명목형 변수 처리하는 방법인디... 그럼 
# 지금 연습문제에서는 일반통계모형이라는건가?

### #04. 0ne-hot encoding방식으로 변환해보기 (in color 변수)

In [ ]:
# 4번은 트리기반 머신러닝에서 명목형 변수 처리하는 연습용이구만 ㅇㅋ
#방법1: colors_ = dfMerged4['color'].values.reshape(-1, 1)
#방법2:
colors_ = dfMerged4[['color']]
colors_.head()

In [ ]:
encoder = OneHotEncoder(sparse_output=False, drop=None) #"first"
# OneHotEncoder는 클래스니까 encoder는 변수가 아니고 객체임!
# -> encoder 객체 안에 어떤 변수랑 프로퍼티들이 있는지 확인하는 방법:
    # dir(encoder)

# fit_transform(): 변수열의 데이터들을 알파벳 순서대로 정렬
#   -> 정렬한 변수들로 컬럼 이름들을 알아서 생성 -> 생성변수들(n개)들에 다
#      0,1의 조합으로 값 지정(n x n matrix 형태로)하는 encoder의 프로퍼티
result = encoder.fit_transform(colors_)
result

In [ ]:
new_cols = encoder.get_feature_names_out(['color'])
# .get_feature_names_out(): 
    # 생성변수의 이름들을 다 출력하는 encoder의 프로퍼티
new_cols

In [ ]:
one_hot_df = DataFrame(
    result, columns=new_cols, index=dfMerged4.index
    )
one_hot_df.head()

In [ ]:
dfMerged4_copy = dfMerged4.copy()
dfMerged5 = merge(dfMerged4_copy, one_hot_df, left_index = True, right_index = True)
dfMerged5.head()

### 5. 데이터 스케일링: 직원 성과 분석을 위한 데이터 스케일링

In [67]:
from hossam import load_data
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler, RobustScaler

In [68]:
origin = load_data('performance')
origin.info()

[data] https://data.hossam.kr/data/lab05/performance.xlsx
[desc] 직원 성과 지표 데이터

field                 description
--------------------  -----------------
EmployeeID            직원 ID
MonthlySales          월별 매출
CustomerSatisfaction  고객 만족도
ProjectHours          프로젝트 작업시간
InnovationScore       혁신 점수

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, E001 to E020
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   MonthlySales          20 non-null     int64
 1   CustomerSatisfaction  20 non-null     int64
 2   ProjectHours          20 non-null     int64
 3   InnovationScore       20 non-null     int64
dtypes: int64(4)
memory usage: 800.0+ bytes


In [69]:
minmax_df = origin.copy()
x1 = minmax_df[['MonthlySales']]
x1.head()

,MonthlySales
EmployeeID,
E001,120
E002,250
E003,80
E004,180
E005,300


In [70]:
scaler1 = MinMaxScaler()
minmax_df['MonthlySales_MinMaxed'] = scaler1.fit_transform(x1)
minmax_df.head()

,MonthlySales,CustomerSatisfaction,ProjectHours,InnovationScore,MonthlySales_MinMaxed
EmployeeID,,,,,
E001,120,75,160,8,0.259259
E002,250,90,180,9,0.740741
E003,80,60,150,6,0.111111
E004,180,82,170,8,0.481481
E005,300,95,190,10,0.925926


In [71]:
maxabs_df = minmax_df.copy()

scaler2 = MaxAbsScaler()

x2 = maxabs_df[['CustomerSatisfaction']]

maxabs_df[['CustomerSatisfaction_MaxAbsed']] = scaler2.fit_transform(x2)

maxabs_df.head()

,MonthlySales,CustomerSatisfaction,ProjectHours,InnovationScore,MonthlySales_MinMaxed,CustomerSatisfaction_MaxAbsed
EmployeeID,,,,,,
E001,120,75,160,8,0.259259,0.765306
E002,250,90,180,9,0.740741,0.918367
E003,80,60,150,6,0.111111,0.612245
E004,180,82,170,8,0.481481,0.836735
E005,300,95,190,10,0.925926,0.969388


In [72]:
std_df = maxabs_df.copy()

scaler3 = StandardScaler()

x3 = std_df[['ProjectHours']]
x4 = std_df[['InnovationScore']]

std_df[['ProjectHours_Stded']] = scaler3.fit_transform(x3)
std_df[['InnovationScore_Stded']] = scaler3.fit_transform(x4)

std_df.head()

,MonthlySales,CustomerSatisfaction,ProjectHours,InnovationScore,MonthlySales_MinMaxed,CustomerSatisfaction_MaxAbsed,ProjectHours_Stded,InnovationScore_Stded
EmployeeID,,,,,,,,
E001,120,75,160,8,0.259259,0.765306,-0.462556,0.066082
E002,250,90,180,9,0.740741,0.918367,0.738889,0.726900
E003,80,60,150,6,0.111111,0.612245,-1.063279,-1.255555
E004,180,82,170,8,0.481481,0.836735,0.138166,0.066082
E005,300,95,190,10,0.925926,0.969388,1.339611,1.387719


In [73]:
std_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, E001 to E020
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   MonthlySales                   20 non-null     int64  
 1   CustomerSatisfaction           20 non-null     int64  
 2   ProjectHours                   20 non-null     int64  
 3   InnovationScore                20 non-null     int64  
 4   MonthlySales_MinMaxed          20 non-null     float64
 5   CustomerSatisfaction_MaxAbsed  20 non-null     float64
 6   ProjectHours_Stded             20 non-null     float64
 7   InnovationScore_Stded          20 non-null     float64
dtypes: float64(4), int64(4)
memory usage: 1.4+ KB


In [74]:
scaled = std_df.filter(
    ['MonthlySales_MinMaxed', 
        'CustomerSatisfaction_MaxAbsed', 
        'ProjectHours_Stded', 
        'InnovationScore_Stded']
    )
scaled

,MonthlySales_MinMaxed,CustomerSatisfaction_MaxAbsed,ProjectHours_Stded,InnovationScore_Stded
EmployeeID,,,,
E001,0.259259,0.765306,-0.462556,0.066082
E002,0.740741,0.918367,0.738889,0.726900
E003,0.111111,0.612245,-1.063279,-1.255555
E004,0.481481,0.836735,0.138166,0.066082
E005,0.925926,0.969388,1.339611,1.387719
E006,0.148148,0.663265,-0.762918,-0.594737
E007,0.555556,0.897959,0.438527,0.726900
E008,0.222222,0.714286,-0.162195,-0.594737
E009,0.629630,0.867347,1.039250,0.726900


In [75]:
t = scaled.T
t

EmployeeID,E001,E002,E003,E004,E005,E006,E007,E008,E009,E010,E011,E012,E013,E014,E015,E016,E017,E018,E019,E020
MonthlySales_MinMaxed,0.259259,0.740741,0.111111,0.481481,0.925926,0.148148,0.555556,0.222222,0.629630,0.074074,0.296296,0.851852,0.037037,0.518519,0.777778,0.185185,0.444444,1.000000,0.000000,0.703704
CustomerSatisfaction_MaxAbsed,0.765306,0.918367,0.612245,0.836735,0.969388,0.663265,0.897959,0.714286,0.867347,0.561224,0.795918,0.938776,0.510204,0.816327,0.928571,0.693878,0.846939,1.000000,0.459184,0.908163
ProjectHours_Stded,-0.462556,0.738889,-1.063279,0.138166,1.339611,-0.762918,0.438527,-0.162195,1.039250,-1.363640,-0.342412,1.219467,-1.664001,0.258311,0.919105,-0.582701,0.018022,1.639972,-1.964363,0.618744
InnovationScore_Stded,0.066082,0.726900,-1.255555,0.066082,1.387719,-0.594737,0.726900,-0.594737,0.726900,-1.255555,0.066082,1.387719,-1.916374,0.066082,0.726900,-0.594737,0.066082,1.387719,-1.916374,0.726900


In [76]:
scaled2 = t.sum()#.T
scaled2

EmployeeID
E001    0.628091
E002    3.124897
E003   -1.595478
E004    1.522464
E005    4.622644
E006   -0.546241
E007    2.618943
E008    0.179576
E009    3.263127
E010   -1.983897
E011    0.815885
E012    4.397813
E013   -3.033134
E014    1.659238
E015    3.352355
E016   -0.298375
E017    1.375487
E018    5.027691
E019   -3.421553
E020    2.957512
dtype: float64

In [ ]:
df = DataFrame(scaled2, index = ['EmployeeID'])
df.index.name = 'EmployeeID'
df
#scaled3 = scaled2.sort_values('EmployeeID', ascending=False)
